<a href="https://colab.research.google.com/github/ishaanpaul98/Sentiment-Analysis/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports #

In [665]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from wordcloud import WordCloud
import re

from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
from nltk.sentiment.vader import SentimentIntensityAnalyzer

import datetime
import yfinance as yf
import time
import requests
import io

In [666]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [667]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

In [668]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,roc_curve,classification_report
from sklearn.metrics import plot_confusion_matrix

# Stock Data Helper Functions #

In [669]:
def getStockDataDaily(symbol, day):
    print("Getting stock data for stock $"+symbol)
    df = yf.download(symbol, start=day, period = "1d")
    return df

getStockDataDaily('AAPL', "2022-12-21")

Getting stock data for stock $AAPL
[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-12-21,132.979996,136.809998,132.750000,135.449997,135.449997,85928000
2022-12-22,134.350006,134.559998,130.300003,132.229996,132.229996,77852100
2022-12-23,130.919998,132.419998,129.639999,131.860001,131.860001,63814900
2022-12-27,131.380005,131.410004,128.720001,130.029999,130.029999,69007800
2022-12-28,129.669998,131.029999,125.870003,126.040001,126.040001,85438400
2022-12-29,127.989998,130.479996,127.730003,129.610001,129.610001,75703700
2022-12-30,128.410004,129.949997,127.430000,129.929993,129.929993,76960600
2023-01-03,130.279999,130.899994,124.169998,125.070000,125.070000,112117500
2023-01-04,126.889999,128.660004,125.080002,126.360001,126.360001,89113600


In [670]:
def arrayToString(arr):
    print("Starting array to list")
    listToStr = ' '.join([str(elem) for elem in arr])
    return listToStr


In [671]:
def getMultiStockDataDaily(symbols, day):
    if len(symbols) > 1:
        print("Length of symbols array is more than 1. STARTING ARRAYTOSTRING")
        symbols = arrayToString(symbols)
    print("Getting stock data for stock $"+symbols)
    df = yf.download(symbols, start=day, period = "1d", group_by='ticker')
    return df

getMultiStockDataDaily(['AAPL', 'TSLA'], "2022-12-21")

Length of symbols array is more than 1. STARTING ARRAYTOSTRING
Starting array to list
Getting stock data for stock $AAPL TSLA
[*********************100%***********************]  2 of 2 completed


AAPL                                                  \
                  Open        High         Low       Close   Adj Close   
Date                                                                     
2022-12-21  132.979996  136.809998  132.750000  135.449997  135.449997   
2022-12-22  134.350006  134.559998  130.300003  132.229996  132.229996   
2022-12-23  130.919998  132.419998  129.639999  131.860001  131.860001   
2022-12-27  131.380005  131.410004  128.720001  130.029999  130.029999   
2022-12-28  129.669998  131.029999  125.870003  126.040001  126.040001   
2022-12-29  127.989998  130.479996  127.730003  129.610001  129.610001   
2022-12-30  128.410004  129.949997  127.430000  129.929993  129.929993   
2023-01-03  130.279999  130.899994  124.169998  125.070000  125.070000   
2023-01-04  126.889999  128.660004  125.080002  126.360001  126.360001   
2023-01-05  127.129997  127.769997  124.760002  125.019997  125.019997   
2023-01-06  126.010002  130.289993  124.889999  129.619995  129.619995   
2023-01-09  130.470001  133.410004  129.889999  130.149994  130.149994   
2023-01-10  130.259995  131.259995  128.119995  130.729996  130.729996   
2023-01-11  131.250000  133.509995  130.460007  133.490005  133.490005   
2023-01-12  133.880005  134.259995  131.440002  133.410004  133.410004   
2023-01-13  132.029999  134.919998  131.660004  134.759995  134.759995   
2023-01-17  134.830002  137.289993  134.130005  135.940002  135.940002   
2023-01-18  136.820007  138.610001  135.029999  135.210007  135.210007   
2023-01-19  134.080002  136.250000  133.770004  135.270004  135.270004   
2023-01-20  135.279999  138.020004  134.220001  137.869995  137.869995   
2023-01-23  138.119995  143.320007  137.899994  141.110001  141.110001   
2023-01-24  140.309998  143.160004  140.300003  142.529999  142.529999   
2023-01-25  140.889999  142.429993  138.809998  141.860001  141.860001   
2023-01-26  143.169998  144.250000  141.899994  143.960007  143.960007   
2023-01-27  143.160004  147.229996  143.080002  145.929993  145.929993   

                             TSLA                                      \
               Volume        Open        High         Low       Close   
Date                                                                    
2022-12-21   85928000  139.339996  141.259995  135.889999  137.570007   
2022-12-22   77852100  136.000000  136.630005  122.260002  125.349998   
2022-12-23   63814900  126.370003  128.619995  121.019997  123.150002   
2022-12-27   69007800  117.500000  119.669998  108.760002  109.099998   
2022-12-28   85438400  110.349998  116.269997  108.239998  112.709999   
2022-12-29   75703700  120.389999  123.570000  117.500000  121.820000   
2022-12-30   76960600  119.949997  124.480003  119.750000  123.180000   
2023-01-03  112117500  118.470001  118.800003  104.639999  108.099998   
2023-01-04   89113600  109.110001  114.589996  107.519997  113.639999   
2023-01-05   80962700  110.510002  111.750000  107.160004  110.339996   
2023-01-06   87686600  103.000000  114.389999  101.809998  113.059998   
2023-01-09   70790800  118.959999  123.519997  117.110001  119.769997   
2023-01-10   63896200  121.070000  122.760002  114.919998  118.849998   
2023-01-11   69458900  122.089996  125.949997  120.510002  123.220001   
2023-01-12   71379600  122.559998  124.129997  117.000000  123.559998   
2023-01-13   57758000  116.550003  122.629997  115.599998  122.400002   
2023-01-17   63646600  125.699997  131.699997  125.019997  131.490005   
2023-01-18   69672800  136.559998  136.679993  127.010002  128.779999   
2023-01-19   58280400  127.260002  129.990005  124.309998  127.169998   
2023-01-20   79972200  128.679993  133.509995  127.349998  133.419998   
2023-01-23   81760300  135.869995  145.380005  134.270004  143.750000   
2023-01-24   66435100  143.000000  146.500000  141.100006  143.889999   
2023-01-25   65799300  141.910004  146.410004  138.070007  144.429993   
2023-01-26   54105100  159.970001  161.419998  

In [672]:
def getMonthlyStockData(symbol, day = datetime.date.today() - datetime.timedelta(days = 1), interval = '1mo'):
    print("Getting stock data for stock $"+symbol)
    df = yf.download(symbol, start=day, period = interval, group_by='ticker')
    return df

#getMonthlyStockData('AAPL', "2022-11-21", '1mo')

# Gathering FinViz Data (Today's News) #

In [673]:
# Parameters 
n = 3 #the # of article headlines displayed per ticker
tickers = ['AAPL']

In [674]:
from finvizfinance.news import News
fnews = News()
all_news = fnews.get_news()
all_news

{'news':        Date                                              Title  \
 0   04:51PM                 Morning Bid: Calm before the storm   
 1   04:22PM  Moving company tried to block negative reviews...   
 2   04:22PM  Apple, Amazon earnings, jobs data and Fed deci...   
 3   04:09PM  Could Big Tech layoffs keep growing? Apple, Am...   
 4   04:00PM  Chinese Travel Is Set to Return. The Question ...   
 ..      ...                                                ...   
 85   Jan-27  S&P 500 jumps to end at nearly 2-month high as...   
 86   Jan-27  : Musk under SEC probe connected with Autopilo...   
 87   Jan-27  FDA approves Eli Lilly's drug for rare blood c...   
 88   Jan-27  China Stocks Veteran Hao Hong Sees Hong Kong S...   
 89   Jan-27  : U.S. oil prices settle at their lowest in mo...   
 
                  Source                                               Link  
 0       www.reuters.com  https://www.reuters.com/markets/asia/global-ma...  
 1       foxbusiness.com  http

In [675]:
# Get Data
finwiz_url = 'https://finviz.com/quote.ashx?t='
news_tables = {}

for ticker in tickers:
    url = finwiz_url + ticker
    print("current url is: " +url)
    header = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36."}
    req = Request(url=url,headers=header) 
    resp = urlopen(req)    
    html = BeautifulSoup(resp, features="lxml")
    news_table = html.find(id='news-table')
    news_tables[ticker] = news_table

try:
    for ticker in tickers:
        df = news_tables[ticker]
        df_tr = df.findAll('tr')
    
        print ('\n')
        print ('Recent News Headlines for {}: '.format(ticker))
        
        for i, table_row in enumerate(df_tr):
            a_text = table_row.a.text
            td_text = table_row.td.text
            td_text = td_text.strip()
            print(a_text,'(',td_text,')')
            if i == n-1:
                break
except KeyError:
    pass

current url is: https://finviz.com/quote.ashx?t=AAPL


Recent News Headlines for AAPL: 
Why This May Be A 'Life Changing' Market Rally; Apple, Fed Meeting Loom As Tesla Run Hits 75% ( Jan-29-23 04:04PM )
Could Big Tech layoffs keep growing? Apple, Amazon, Facebook and Google may give hints in biggest week of earnings ( 03:01PM )
15 Most Famous Hedge Fund Managers and Their Top Stock Picks ( 02:35PM )


In [676]:
# Iterate through the news
parsed_news = []
for file_name, news_table in news_tables.items():
    for x in news_table.findAll('tr'):
        print(x.get_text())
        text = x.get_text() 
        date_scrape = x.td.text.split()

        if len(date_scrape) == 1:
            time = date_scrape[0]
            
        else:
            date = date_scrape[0]
            time = date_scrape[1]

        ticker = file_name.split('_')[0]
        
        parsed_news.append([ticker, date, time, text ])

Jan-29-23 04:04PMWhy This May Be A 'Life Changing' Market Rally; Apple, Fed Meeting Loom As Tesla Run Hits 75% Investor's Business Daily
03:01PMCould Big Tech layoffs keep growing? Apple, Amazon, Facebook and Google may give hints in biggest week of earnings MarketWatch
02:35PM15 Most Famous Hedge Fund Managers and Their Top Stock Picks Insider Monkey
10:30AMTarget, Amazon and 4 More Retailers That Will Reward You for Turning in Your Old Stuff GOBankingRates
07:53AMFed meeting, jobs data, Apple earnings: What to know this week Yahoo Finance

07:00AM
Loading…

07:00AM$50 AirPods Pro? Nope. Heres How to Spot Fake Apple Earbuds. The Wall Street Journal
06:00AMApple (NASDAQ:AAPL) stock performs better than its underlying earnings growth over last five years Simply Wall St.
05:45AM5 Top Stocks for February Motley Fool
01:09AMChina's 2022 smartphone shipments the lowest in 10 years - research firm Reuters
01:00AMChina's 2022 smartphone shipments the lowest in 10 years - research firm Reuters

# Gathering Data From AlphaAdvantage for Historical News #

In [677]:
from decouple import config
import requests
import urllib.parse
import json
import datetime
AAapikey = config('AAKey')

In [678]:
#Function to convert user provided date to date required by AlphaAdvantage
def toAADate(oldDate):
    newDate = oldDate.strftime("%Y%m%dT0130")
    return str(newDate)


In [679]:
# Get data from AlphaAdvantage for one ticker for a particular day
def getHistoricNewsData(ticker, endDate = datetime.date.today() - datetime.timedelta(days = 1) , interval = '1mo'):
    url = 'https://www.alphavantage.co/query?'
    print("endDate is: " +str(endDate))
    if interval == '1mo':
        days_to_add = 30
    else:
        days_to_add = 60
    delta_days = datetime.timedelta(days = days_to_add)
    startDate = endDate - delta_days
    print("startDate is: " +str(startDate))
    if startDate and endDate:
        startDate = toAADate(startDate)
        endDate = toAADate(endDate)
        print("startDate is: " +str(startDate))
        print("endDate is: " +str(endDate))
        Myparams = {'function': 'NEWS_SENTIMENT', 'tickers': ticker, 'time_from': startDate, 'time_to': endDate, 'sort': 'EARLIEST','limit': 200, 'apikey': AAapikey}
    else:
        print("NEED DATES")
        #Myparams = {'function': 'NEWS_SENTIMENT', 'tickers': ticker, 'sort': 'LATEST','limit': 100, 'apikey': AAapikey}
    r = requests.get(url, params = Myparams)
    data = r.json()
    #return data
    historic_news = pd.DataFrame(columns=['Date', 'Headline', 'Ticker'])
    for i in data.get("feed"):
        test_date = i.get("time_published")
        test_date = test_date[:8]
        newDate = datetime.datetime.strptime(test_date, '%Y%m%d').date()
        row = [newDate, i.get("title"), ticker]
        new_df = pd.DataFrame([row],columns=['Date', 'Headline', 'Ticker'])
        historic_news = pd.concat([historic_news, new_df], axis=0, ignore_index=True)
    return historic_news

#historic_news = getHistoricNewsData('AAPL', '2022-10-10', '2mo')
historic_news = getHistoricNewsData('AAPL', interval = '2mo')
print(historic_news)

endDate is: 2023-01-28
startDate is: 2022-11-29
startDate is: 20221129T0130
endDate is: 20230128T0130
           Date                                           Headline Ticker
0    2022-11-29   Asia shares take comfort in China property rally   AAPL
1    2022-11-29  Asia shares take comfort in China property ral...   AAPL
2    2022-11-29  Global markets: Asia shares take comfort in Ch...   AAPL
3    2022-11-29  App Store Awards Celebrate the Best Apps and G...   AAPL
4    2022-11-29  iPhone 15 Might Push HD Photography To New Hei...   AAPL
..          ...                                                ...    ...
195  2022-12-05  Foxconn says it's restoring production at the ...   AAPL
196  2022-12-05  The 3 Most Popular Robinhood Stocks Right Now:...   AAPL
197  2022-12-05  US Stocks Start New Trading Week On Negative N...   AAPL
198  2022-12-05  Apple In A Rush To Diversify iPhone Production...   AAPL
199  2022-12-05  Should SPDR S&P 500 ETF  ( SPY )  Be on Your I...   AAPL

[200 rows

# Sentiment Analysis of News data #

In [682]:
def SentimentAnalysisNewsData(parsedNews, printOut = False):
    #Downloading Vader Lexicon for Sentiment Analysis
    nltk.download('vader_lexicon')
    # Initializing Sentiment Analysis
    analyzer = SentimentIntensityAnalyzer()

    #Declaring Column Names
    columns = ['Ticker', 'Date', 'Time', 'Headline']
    #Creating dataframe from news
    news = pd.DataFrame(parsedNews, columns=columns)
    #Getting scores for headlines
    scores = news['Headline'].apply(analyzer.polarity_scores).tolist()

    #Creating Dataframe of Scores
    df_scores = pd.DataFrame(scores)
    #Joining scores to news dataframe
    news = news.join(df_scores, rsuffix='_right')
    #Converting Date column to pd datetime date
    news['Date'] = pd.to_datetime(news.Date).dt.date

    #List of unique tickers
    unique_ticker = news['Ticker'].unique().tolist()
    #Creating dict for news based on ticker
    news_dict = {name: news.loc[news['Ticker'] == name] for name in unique_ticker}
    #Initializing List of values
    values = []
    for ticker in tickers: 
        dataframe = news_dict[ticker]
        dataframe = dataframe.set_index('Ticker')
        #Dropping headlines column since we only need scores now
        dataframe = dataframe.drop(columns = ['Headline'])
        #if printOut:
            #print ('\n')
            #print (dataframe.head())
        
        #mean = round(dataframe['compound'].mean(), 2)
        #Finding compound number for news of every day
        mean = round(dataframe.groupby('Date')['compound'].mean(), 2)
        #Adding values to values list
        values.append(mean)
    
   
    #print(round(dataframe.groupby('Date')['compound'].mean(), 2))
    print("VALUES------------")
    print(values)
        
    #Combining tickers and values into new dataframe
    #df = pd.DataFrame(list(zip(tickers, values)), columns =['Ticker', 'Mean Sentiment']) 
    #df = df.set_index('Ticker')
    #df = df.sort_values('date', ascending=False)
    if printOut:
        print("-----------DF")
        print(df.head())
        print(df.shape)
    #Returning the dataframe
    return df
    #if printOut:
        #print ('\n')
        #display (df)
    #return df

In [681]:
#print("HISTORIC SENTIMENT")
HistoricSentiment = SentimentAnalysisNewsData(historic_news, True)
#print("\n")
#print("TODAYS SENTIMENT")
#TodaysSentiment = SentimentAnalysisNewsData(parsed_news)

VALUES------------
[Date
2022-11-29    0.16
2022-11-30    0.03
2022-12-01    0.04
2022-12-02    0.12
2022-12-03   -0.06
2022-12-04   -0.03
2022-12-05   -0.02
Name: compound, dtype: float64]
-----------DF


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Ishaan\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


TypeError: 'NoneType' object is not callable

# Creating Dataset #

In [ ]:
def createDataset(date_from, int):
    for i in tickers:
        print("Creating dataset for $" +i)
        #Get historic stock data
        historic_stock = getMonthlyStockData(i, interval = int)
        print(historic_stock.head())
        #Get historic news data
        historic_news = getHistoricNewsData(i, interval = int)
        print(historic_news)
        #Use news to get sentiment
        HistoricSentiment = SentimentAnalysisNewsData(historic_news)
        print(HistoricSentiment.shape)
        #Merge as training set
        #Get today's stock data
        #Get today's news data
        #Use news to get sentiment


createDataset('2022-10-10', '2mo')

Creating dataset for $AAPL
Getting stock data for stock $AAPL
[*********************100%***********************]  1 of 1 completed
                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2023-01-25  140.889999  142.429993  138.809998  141.860001  141.860001   
2023-01-26  143.169998  144.250000  141.899994  143.960007  143.960007   

              Volume  
Date                  
2023-01-25  65799300  
2023-01-26  54003800  
endDate is: 2023-01-25
startDate is: 2022-11-26
startDate is: 20221126T0130
endDate is: 20230125T0130
           Date                                           Headline Ticker
0    2022-11-26  10 exchange-traded funds to take exposure in U...   AAPL
1    2022-11-26  Market Rally Strong, But Here's Why You Should...   AAPL
2    2022-11-26  Motley Fool Investors Look Back at 2022 and Fo...   AAPL
3    2022-11-26  Black Friday Online Sales Hit Record, But Grow...   AAPL
4    202

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Ishaan\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
